# Import Necessary Packages

In [1]:
!pip install wbdata

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install prophet


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from datetime import datetime, date
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import altair as alt
import ipywidgets as widgets
from ipywidgets import VBox
from IPython.display import display, clear_output
from IPython.display import HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import wbdata
# from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly
import logging
from ipywidgets import Layout
import warnings

# 股票分析看板

In [4]:
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

# Function to download stock data
def download_stock_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, progress=False)['Close']

# Function to plot stock data
def plot_stock_data(stock_1_data, stock_1_ticker, stock_2_data=None, stock_2_ticker=None):
    fig = make_subplots(specs=[[{"secondary_y": stock_2_data is not None}]])
    fig.add_trace(
        go.Scatter(
            x=stock_1_data.index,
            y=stock_1_data,
            mode='lines',
            name=stock_1_ticker,  # Use ticker as legend name
            line=dict(color='#1f77b4')
        ),
        secondary_y=False
    )
    
    if stock_2_data is not None:
        fig.add_trace(
            go.Scatter(
                x=stock_2_data.index,
                y=stock_2_data,
                mode='lines',
                name=stock_2_ticker,  # Use ticker as legend name
                line=dict(color='#ff7f0e')
            ),
            secondary_y=True
        )

    fig.update_layout(
        title="Line Chart of Close Prices",
        template="plotly_white",
        xaxis_title="Date",
        yaxis_title="Close Price"
    )
    fig.show()
    
# Function to display analysis results
def display_results(stock_ticker_1, stock_ticker_2, correlation, beta, volatility_1, volatility_2):
    results_html = f"""
    <table>
    <tr><th>Analysis</th><th>{stock_ticker_1}</th><th>{stock_ticker_2}</th></tr>
    <tr><td>Pearson Correlation between {stock_ticker_1} and {stock_ticker_2}</td><td colspan='2'>{correlation:.4f}</td></tr>
    <tr><td>Beta Coefficient of {stock_ticker_1} relative to {stock_ticker_2}</td><td colspan='2'>{beta:.4f}</td></tr>
    <tr><td>Volatility</td><td>{volatility_1:.4f}</td><td>{volatility_2:.4f}</td></tr>
    </table>
    """
    display(HTML(results_html))

# Function for financial analysis
def perform_financial_analysis(stock_1_data, stock_1_ticker, stock_2_data, stock_2_ticker):
    aligned_data = pd.merge(stock_1_data.reset_index(), stock_2_data.reset_index(), on='Date', suffixes=('_1', '_2'))
    
    # Pearson Correlation
    correlation = aligned_data.iloc[:, 1].corr(aligned_data.iloc[:, 2])
    
    # Beta Coefficient
    covariance = np.cov(aligned_data.iloc[:, 1].pct_change().dropna(), aligned_data.iloc[:, 2].pct_change().dropna())[0][1]
    variance = aligned_data.iloc[:, 2].pct_change().var()
    beta = covariance / variance
    
    # Volatility Comparison
    volatility_1 = aligned_data.iloc[:, 1].pct_change().std()
    volatility_2 = aligned_data.iloc[:, 2].pct_change().std()
    
    # Display results
    display_results(stock_1_ticker, stock_2_ticker, correlation, beta, volatility_1, volatility_2)
    
# New function for prediction
def predict_stock_price(stock_data, periods=30):
    df = stock_data.reset_index()
    df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)  # Assuming 'Close' is the column with stock prices
    
    model = Prophet()
    model.fit(df)
    
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    
    return forecast, model
    
# Function to display prediction properties
def display_prediction_properties(model, forecast):
    fig = make_subplots(rows=3, cols=1)
    # Trend
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend'), row=1, col=1)
    
    # Yearly
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yearly'], name='Yearly Seasonality'), row=2, col=1)
    
    # Weekly
    if 'weekly' in forecast:
        fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['weekly'], name='Weekly Seasonality'), row=3, col=1)
    
    fig.update_layout(height=600, width=800, title_text="Prophet Model Components")
    fig.show()
    
def evaluate_prophet_model(model, df, start_date, end_date):
    # Calculate the total days of data available
    total_days = (end_date - start_date).days
    
    # Ensure there's enough data for initial training period and horizon
    if total_days <= 730:  # Less than or equal to 2 years
        print("Limited data available for evaluation. Adjusting validation strategy.")
        # Adjust 'initial' and 'horizon' based on available data
        initial_days = max(180, total_days // 3)  # Use a minimum of 6 months or a third of total days
        horizon_days = max(30, total_days // 6)  # Use a minimum of 1 month or a sixth of total days
    else:
        # Set 'initial' to half of the data's timeframe up to a maximum of 3 years (in days)
        initial_days = min(1095, total_days // 2)  # 3 years in days or half of the total days
        horizon_days = 365  # Set 'horizon' to 1 year (365 days)
    
    initial = f'{initial_days} days'
    horizon = f'{horizon_days} days'
    
    # Set 'period' based on the adjusted 'initial' period to ensure multiple evaluations
    period_days = max(90, initial_days // 4)  # At least 3 months, a quarter of the initial
    period = f'{period_days} days'
    
    try:
        # Perform cross-validation
        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)
        
        # Calculate performance metrics
        df_p = performance_metrics(df_cv)
        
        # Display evaluation metrics
        display_evaluation_metrics(df_p)
    except Exception as e:
        print(f"Error during model evaluation: {e}")

def display_evaluation_metrics(df_p):
    metrics = df_p.mean().to_dict()  # Averaging performance metrics for simplicity
    
    metrics_html = "<div><h3>Model Evaluation Metrics</h3><table border='1'><tr>"
    metrics_html += "".join([f"<th>{metric}</th>" for metric in metrics.keys()])
    metrics_html += "</tr><tr>"
    
    # Adjust formatting to handle Timedelta and numeric values separately
    for value in metrics.values():
        if isinstance(value, pd.Timedelta):
            # Convert Timedelta to days, for example
            formatted_value = f"{value.total_seconds() / 86400:.2f} days"
        else:
            # Assume value is numeric and format as before
            formatted_value = f"{value:.4f}"
        metrics_html += f"<td>{formatted_value}</td>"
    
    metrics_html += "</tr></table></div>"
    display(HTML(metrics_html))
    
# Button click event handler for prediction
def on_predict_button_clicked(b):
    with output:
        clear_output(wait=True)
        start_date_val = pd.to_datetime(start_date.value)
        end_date_val = pd.to_datetime(end_date.value)
        stock_data = download_stock_data(stock_ticker_1.value, start_date_val, end_date_val)
        forecast, model = predict_stock_price(stock_data, periods=30)
        
        # Use Plotly for interactive forecast plot
        fig = plot_plotly(model, forecast)
        fig.update_layout(title="Stock Price Forecast")
        fig.show()

        # Optionally, use Plotly for interactive components plot
        fig_components = plot_components_plotly(model, forecast)
        fig_components.update_layout(title="Prophet Model Components")
        fig_components.show()
        
        # Evaluate and display metrics horizontally
        evaluate_prophet_model(model, stock_data, start_date_val, end_date_val)
        
# The main function to fetch data, plot, and calculate financial metrics
def fetch_plot_correlation(stock_ticker_1, stock_ticker_2, start_date, end_date):
    stock_1_data = download_stock_data(stock_ticker_1, start_date, end_date)
    stock_2_data = download_stock_data(stock_ticker_2, start_date, end_date) if stock_ticker_2 else None
    
    plot_stock_data(stock_1_data, stock_ticker_1, stock_2_data, stock_ticker_2)
    
    if stock_ticker_2:
        perform_financial_analysis(stock_1_data, stock_ticker_1, stock_2_data, stock_ticker_2)

# Define the button click event handler
def on_button_clicked(b):
    with output:
        clear_output(wait=True)  # Clear the output before displaying new content
        ticker_2 = stock_ticker_2.value if analysis_mode.value == 'Multiple Stock' else None
        fetch_plot_correlation(stock_ticker_1.value, ticker_2, start_date.value.strftime('%Y-%m-%d'), end_date.value.strftime('%Y-%m-%d'))

# Layout adjustment based on the analysis mode
def adjust_layout(*args):
    if analysis_mode.value == 'Single Stock':
        stock_ticker_2.layout.display = 'none'
        predict_button.layout.display = 'flex'
    else:
        stock_ticker_2.layout.display = 'flex'
        predict_button.layout.display = 'none'

        
# Prepare UI components
dashboard_heading = widgets.HTML(value="<h2 style='text-align: left; color: #c71306;'>股票分析看板</h2>")
analysis_mode = widgets.Dropdown(options=['Single Stock', 'Multiple Stock'], value='Multiple Stock', description='Mode:')
stock_ticker_1 = widgets.Text(value='000001.SS', description='Ticker 1:', disabled=False)
stock_ticker_2 = widgets.Text(value='^HSI', description='Ticker 2:', disabled=False)
start_date = widgets.DatePicker(description='Start Date', value=pd.to_datetime('2010-01-01'))
end_date = widgets.DatePicker(description='End Date', value=pd.to_datetime(datetime.now().strftime('%Y-%m-%d')))
run_button = widgets.Button(
    description='Price Chart',
    layout=Layout(width='auto', height='auto', justify_content='center'))
predict_button = widgets.Button(
    description='Time Series Analysis',
    layout=Layout(width='auto', height='auto', justify_content='center')
)
predict_button.layout.display = 'none'  # Initially hidden

output = widgets.Output()

predict_button.on_click(on_predict_button_clicked)
run_button.on_click(on_button_clicked)

analysis_mode.observe(adjust_layout, 'value')
adjust_layout()  # Adjust layout initially based on the default mode

# Modify the display line to include the predict_button
display(dashboard_heading, analysis_mode, stock_ticker_1, stock_ticker_2, start_date, end_date, run_button, predict_button, output)

HTML(value="<h2 style='text-align: left; color: #c71306;'>股票分析看板</h2>")

Dropdown(description='Mode:', index=1, options=('Single Stock', 'Multiple Stock'), value='Multiple Stock')

Text(value='000001.SS', description='Ticker 1:')

Text(value='^HSI', description='Ticker 2:', layout=Layout(display='flex'))

DatePicker(value=Timestamp('2010-01-01 00:00:00'), description='Start Date')

DatePicker(value=Timestamp('2024-03-17 00:00:00'), description='End Date')

Button(description='Price Chart', layout=Layout(height='auto', justify_content='center', width='auto'), style=…

Button(description='Time Series Analysis', layout=Layout(display='none', height='auto', justify_content='cente…

Output()